In [1]:
import pandas as pd
from collections import defaultdict
from datatypes import ClimateHealthTimeSeries

ModuleNotFoundError: No module named 'climate_health.datatypes'; 'climate_health' is not a package

In [29]:
def lagged_rows(data, lag_rows, lag=1):
    new_data = data.iloc[lag:]
    for lag_row in lag_rows:
        for i in range(lag):
            lagged_col = data.iloc[lag-i-1:-1-i, lag_row].values
            new_data.insert(len(new_data.columns), f"{data.columns[lag_row]}_lag_{i+1}", lagged_col)
    return new_data

In [30]:
def split_to_train_test_truth_fixed_ahead_lag(data, split_fraction):
    hardcoded_test_col = 3
    n = data.shape[0]
    train_idx = n * split_fraction % 1
    X = data.drop(data.columns[hardcoded_test_col], axis = 1) 
    Y = data.iloc[:, hardcoded_test_col]

    X_train = X.iloc[:train_idx]
    X_test = X.iloc[train_idx:]
    Y_train = Y.iloc[:train_idx]
    Y_test = Y.iloc[train_idx:]

    return X_train, Y_train, X_test, Y_test

In [31]:
class PlaceholderModel:

    def __init__(self):
        return

    def fit(self, X_train, Y_train):
        return self
    
    def predict(self, single_X_test):
        return 0

In [26]:
class AssessmentReport:
    def __init__(self, text):
        self.text = text
        return

In [27]:
def make_assessment_report(prediction_dict, truth_dict) -> AssessmentReport:
    return AssessmentReport("Good job!")

In [5]:
nice_data = pd.read_csv("../../example_data/data.csv")

In [3]:
lags = [1, 5, 10]
models = [ch.Regressor(...), ch.Guesser(...)]

In [ ]:
def main_with_outer_lagged(fn: str, split_fraction: float, model: ChPredictionMethodWithSpecificFeatureConfig) -> AssessmentReport:
    data = ClimateHealthTimeSeries.from_csv(fn)  # real data or simulated data
    data = data.topandas()
    
    prediction_dict = defaultdict(dict)
    truth_dict = {}
    for lag_ahead in range(1, 10):
        rowbased_data = lagged_rows(data, lag_rows=[3], lag=lag_ahead)
        X_train, Y_train, X_test, Y_test = split_to_train_test_truth_fixed_ahead_lag(rowbased_data, now_timepoint, lag_ahead)
        model.fit(X_train, Y_train)
        for test_time_offset, (single_X_test, single_Y_test) in enumerate(zip(X_test, Y_test)):
            prediction_dict[lag_ahead][test_time_offset] = model.predict(single_X_test)
            truth_dict[lag_ahead][test_time_offset] = single_Y_test   
    report = make_assessment_report(prediction_dict, truth_dict)
    return report

In [ ]:
main_with_outer_lagged()

In [ ]:
import climate_health as ch
import pandas as pd

#nice_data = ch.simulate(corr = 0.6, length = 3000)
nice_data = pd.read_csv("nice_data.csv")

lags = [1, 5, 10]
models = [ch.Regressor(...), ch.Guesser(...)]

scores = MVP(nice_data, lags, models, time_point)

# MSE    - lag - mdl
# 0.3252 -  1  - Regressor
# 0.4552 -  5  - Regressor
# 0.7252 - 10  - Regressor
# 0.8252 -  1  - Guesser
# 0.9252 -  5  - Guesser
# 0.8652 - 10  - Guesser

In [ ]:
def MVP(nice_data, lags, models, time_point):
    for lag in lags:
        lagged_data = ch.lag(nice_data, lag)
        train, test = ch.split(lagged_data, time_point)
        for model in models:
            my_mdl = model.train(train)
            score = my_mdl.test(test)
            print(score, lag, model)

In [89]:
import pandas as pd
import matplotlib.pyplot as plt

In [166]:
data = pd.read_csv("../../example_data/data.csv")

In [9]:
new_data = lagged_rows(nice_data, lag_rows = [3], lag=2)

In [10]:
new_data

,Time,Rain,Temperature,Disease,Disease_lag_1,Disease_lag_2
2,2003-01-15,0.000000,17.664000,84.880000,83.200000,88.000000
3,2003-01-22,0.000000,17.397600,87.892000,84.880000,83.200000
4,2003-01-29,0.000000,17.157840,92.102800,87.892000,84.880000
5,2003-02-05,0.000000,16.223874,100.983429,92.102800,87.892000
6,2003-02-12,0.000000,16.283305,109.566904,100.983429,92.102800
...,...,...,...,...,...,...
1091,2023-11-29,15.636364,20.708580,692.818869,703.234498,710.811459
1092,2023-12-06,15.636364,21.037722,680.757530,692.818869,703.234498
1093,2023-12-13,0.000000,20.433950,668.587780,680.757530,692.818869
1094,2023-12-20,0.000000,18.990555,663.635005,668.587780,680.757530
